<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=500 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Heart Disease Prediction using Hidden Markov Models  <br>
<font color=696880 size=4>
Omid Daliran


# Personal Data

In [2]:
# Set your student number and name
student_number = "401105815"
Name = "Radin"
Last_Name = "Cheraghi"

# Libraries

In [3]:
import csv
import numpy as np
import pandas as pd

# Q2: Heart Disease Prediction using Hidden Markov  (100 Points)

# Introduction


In this notebook, we explore the application of Hidden Markov Models (HMM) in predicting heart disease risk based on DNA sequences. Heart disease is a prevalent and life-threatening condition, and early detection is crucial for effective management. By leveraging HMM, we aim to identify regions within DNA sequences associated with high and low GC content, which have been linked to heart disease susceptibility.


# Hidden Markov Models (HMM) (40 points)

Hidden Markov Models are probabilistic models widely used in sequential data analysis, particularly in bioinformatics. They are characterized by states, transitions between states, and emission probabilities associated with each state. In our context, the states represent the underlying biological characteristics of DNA sequences, while the emission probabilities signify the likelihood of observing specific nucleotides given each state.

To facilitate our heart disease classification, we'll implement a custom Hidden Markov Model (HMM) class in Python. This class will include the Viterbi and Forward algorithms.

In [16]:
class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list):
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities

    def viterbi(self, sequence: str) -> tuple:
        '''
        This method performs the Viterbi algorithm within the Hidden Markov Model (HMM) class to predict the most likely sequence of hidden states given an observed sequence, such as a DNA sequence.

        Parameters:
        - sequence: The observed sequence for which the most likely hidden state sequence is to be predicted.

        Returns:
        - path: The predicted sequence of hidden states corresponding to the input sequence.
        - max_probability: The maximum probability associated with the predicted path.
        '''
        # TODO
        num_of_states = len(self.states)
        num_of_emissions = len(sequence)
        viterbi_matrix = [[0] * num_of_states for i in range(num_of_emissions)]
        path = [[0] * num_of_states for i in range(num_of_emissions)]
        for i in range(num_of_states):
          viterbi_matrix[0][i] = self.start_probabilities[i] * self.emissions[i][sequence[0]]
          path[0][i] = i
        for i in range(1,num_of_emissions):
          for j in range(num_of_states):
            max_tmp = 0
            for k in range(num_of_states):
              #print(viterbi_matrix[i-1][k], self.transition_probabilities[k][j], self.emissions[j][sequence[i]])
              if viterbi_matrix[i-1][k] * self.transition_probabilities[k][j] * self.emissions[j][sequence[i]] > max_tmp:
                max_tmp = viterbi_matrix[i-1][k] * self.transition_probabilities[k][j] * self.emissions[j][sequence[i]]
                path[i][j] = k
                viterbi_matrix[i][j] = max_tmp
        max_probability = 0
        end_state = None
        for k in range(num_of_states):
          if max_probability < viterbi_matrix[num_of_emissions - 1][k]:
            max_probability = viterbi_matrix[num_of_emissions - 1][k]
            end_state = k
        # print(viterbi_matrix)
        final_path = [end_state]
        for i in range(num_of_emissions - 1, 0 , -1):
          final_path = [path[i][end_state]] + final_path
          end_state = path[i][end_state]
        path = ""
        for i in final_path:
          path += self.state_labels[i]
        return path, max_probability

    def forward_algorithm(self, sequence: str) -> list:
        '''
        This method performs the Forward algorithm within the Hidden Markov Model (HMM) class to calculate the probability of observing a given sequence under the model.

        Parameters:
        - sequence: The observed sequence for which the likelihood is to be calculated.

        Returns:
        - probability_last_states: The probabilities of the final states after observing the entire sequence.
        '''
        # TODO
        num_of_states = len(self.states)
        num_of_emissions = len(sequence)
        forward_matrix = [[0.0] * num_of_states for i in range(num_of_emissions)]
        for i in range(num_of_states):
          forward_matrix[0][i] = self.start_probabilities[i] * self.emissions[i][sequence[0]]
        for i in range(1,num_of_emissions):
          for j in range(num_of_states):
            sum = 0
            for k in range(num_of_states):
              sum += forward_matrix[i-1][k] * self.transition_probabilities[k][j] * self.emissions[j][sequence[i]]
            forward_matrix[i][j] = sum
        sum = 0
        for i in range(num_of_states):
          sum += forward_matrix[num_of_emissions - 1][i]
        for i in range(num_of_states):
          forward_matrix[num_of_emissions - 1][i] /= sum
        probability_last_states = forward_matrix[num_of_emissions - 1]

        return probability_last_states



GC content, the proportion of guanine (G) and cytosine (C) nucleotides in a DNA sequence, is a fundamental genomic feature. Variations in GC content have been implicated in various biological processes, including gene regulation and disease susceptibility. Specifically, high GC content (H) regions are associated with increased gene expression and potential regulatory elements, while low GC content (L) regions may indicate structural elements or regions prone to mutations. The transition and emission and start probabilities are as shown in the image below:

![Probabilities](Probabilities.png)

In [17]:
# TODO: Complete and run this code to check your implementation

states = ['H', 'L']
emissions = [{'A' : 0.2, 'C' : 0.3, 'G' : 0.3, 'T' : 0.2},
             {'A' : 0.3, 'C' : 0.2, 'G' : 0.2, 'T' : 0.3}]
start_probabilities = [0.5, 0.5]
transition_probabilities = [[0.5 , 0.5] , [0.4 , 0.6]]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

# Example DNA sequence
sequence = "ATGCGCGATCGATCGAATCGCGT"

# Viterbi algorithm
path, max_probability = hmm.viterbi(sequence)
print("Viterbi Path:", path)
print("Max Probability (Viterbi):", max_probability)

# Forward algorithm
probability = hmm.forward_algorithm(sequence)
print("Probability (Forward Algorithm):", probability)

Viterbi Path: LLHHHHHLLHHLLHHLLLHHHHL
Max Probability (Viterbi): 1.1438396227480495e-19
Probability (Forward Algorithm): [0.3581800885349868, 0.6418199114650132]


# Calculating Probabilities for Blood Type, Height, and Weight (10 points)

After obtaining the sequence of H and L regions from the DNA using the Viterbi algorithm, we proceed to calculate the probabilities for other attributes such as blood type, height, weight, and the presence or absence of DCM (DCM, or Dilated Cardiomyopathy, is a heart condition characterized by the enlargement of the heart's left ventricle. It can lead to heart failure if not properly managed.) using the Forward algorithm. In this step, we aim to leverage the Hidden Markov Model (HMM) to infer these attributes based on the observed GC content sequence.


We will use the functionality of the Forward algorithm to compute the probabilities associated with each possible state after observing the sequence.

Attributes and Possible States:
* Blood Type: O, A, B, AB
* Height: 160 cm, 170 cm, 180 cm, 190 cm
* Weight: 50 kg, 65 kg, 80 kg, 95 kg
* DCM: Y (indicating presence) or N (indicating absence)


In [25]:
def probabilities_from_sequence(sequence: str) -> tuple:
  '''
  This function, `probabilities_from_sequence(sequence)`, computes the probabilities associated with attributes such as blood type, height, weight, and the presence or absence of Dilated Cardiomyopathy (DCM) based on an input DNA sequence.

  ### Input:
  - `sequence`: The observed DNA sequence for which probabilities are to be computed.

  ### Output:
  The function returns dictionaries containing the probabilities for each possible state of the respective attributes:
  - `height_probability_dic`: Probabilities for height categories (160 cm, 170 cm, 180 cm, 190 cm).
  - `weight_probability_dic`: Probabilities for weight categories (50 kg, 65 kg, 80 kg, 95 kg).
  - `blood_type_probability_dic`: Probabilities for blood types (O, A, B, AB).
  - `DCM_probability_dic`: Probabilities for the presence (Y) or absence (N) of Dilated Cardiomyopathy (DCM).
  '''
  # Define the emission probabilities for each feature

  height_emissions = [{'H': 0.2, 'L': 0.8}, {'H': 0.3, 'L': 0.7},
                      {'H': 0.9, 'L': 0.1}, {'H': 0.8, 'L': 0.2}]

  weight_emissions = [{'H': 0.3, 'L': 0.7}, {'H': 0.1, 'L': 0.9},
                      {'H': 0.2, 'L': 0.8}, {'H': 0.7, 'L': 0.3}]

  blood_type_emissions = [{'H': 0.75, 'L': 0.25}, {'H': 0.8, 'L': 0.2},
                          {'H': 0.85, 'L': 0.15}, {'H': 0.7, 'L': 0.3}]

  DCM_emissions = [{'H': 0.7, 'L': 0.3}, {'H': 0.6, 'L': 0.4}]

  # Define the transition probabilities for each feature
  height_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
  weight_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
  blood_type_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
  DCM_transition_probabilities = [[0.9, 0.1], [0.1, 0.9]]


  # Define start probabilities for each state
  height_start_probabilities = [0.1, 0.4, 0.4, 0.1]
  weight_start_probabilities = [0.15, 0.35, 0.35, 0.15]
  blood_type_start_probabilities = [0.1, 0.35, 0.35, 0.2]
  DCM_start_probabilities = [0.25, 0.75]


  height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic = {}, {}, {}, {}


  # TODO: complete the code
  hmm1 = HMM(['160', '170', '180', '190'], height_emissions, height_start_probabilities, height_transition_probabilities)
  hmm2 = HMM(['50', '65', '80', '95'], weight_emissions, weight_start_probabilities, weight_transition_probabilities)
  hmm3 = HMM(['O', 'A', 'B', 'AB'], blood_type_emissions, blood_type_start_probabilities, blood_type_transition_probabilities)
  hmm4 = HMM(['Y', 'N'], DCM_emissions, DCM_start_probabilities, DCM_transition_probabilities)
  list1 = hmm1.forward_algorithm(sequence)
  index = 0
  for state in ['160', '170', '180', '190']:
    height_probability_dic[int(state)] = list1[index]
    index += 1
  list2 = hmm2.forward_algorithm(sequence)
  index = 0
  for state in ['50', '65', '80', '95']:
    weight_probability_dic[int(state)] = list2[index]
    index += 1
  list3 = hmm3.forward_algorithm(sequence)
  index = 0
  for state in ['O', 'A', 'B', 'AB']:
    blood_type_probability_dic[state] = list3[index]
    index += 1
  list4 = hmm4.forward_algorithm(sequence)
  index = 0
  for state in ['Y', 'N']:
    DCM_probability_dic[state] = list4[index]
    index += 1
  return height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic



In [26]:
# Run this code to check your implementation
# Example sequence of 'H' and 'L'
sequence = "HHLLHHHLLHLLLLHHLL"

height_probability, weight_probability, blood_type_probability, DCM_probability = probabilities_from_sequence(sequence)

print("Probability of height given sequence:", height_probability)
print("Probability of weight given sequence:", weight_probability)
print("Probability of blood type given sequence:", blood_type_probability)
print("Probability of DCM given sequence:", DCM_probability)

Probability of height given sequence: {160: 0.36605931748711135, 170: 0.5760951509787055, 180: 0.015850353367627773, 190: 0.04199517816655549}
Probability of weight given sequence: {50: 0.2845952908892867, 65: 0.42707075390249144, 80: 0.20957356322963908, 95: 0.07876039197858277}
Probability of blood type given sequence: {'O': 0.29628284733090365, 'A': 0.3305080573190477, 'B': 0.08789883086305705, 'AB': 0.2853102644869917}
Probability of DCM given sequence: {'Y': 0.3297758181008331, 'N': 0.6702241818991669}


# Bayesian Network for Heart Disease Probability Calculation (30 points)

In this section, we introduce a Bayesian network to calculate the probability of heart disease based on various features. We assume that the relationships between features follow a specific structure, depicted in the image below.

![Bayesian Net](BN.png)

The Bayesian network comprises nodes representing different features, including sex, weight, height, Dilated Cardiomyopathy (DCM), blood type, and heart disease. The directed edges between nodes indicate probabilistic dependencies between them.

Function Implementation:
We will implement two functions to facilitate the calculation of heart disease probability:

1. calculate_sex_probability:

  This function calculates the probability of sex based on the given probabilities of weight and height.

2. calculate_heart_disease_probability:

  This function computes the probability of heart disease based on the probabilities of sex, DCM, and blood type.

In [31]:
# The probability of sex given height and weight, the tuple is in this order: (sex, height, weight)
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}

In [32]:
def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    '''
    This function, `calculate_sex_probability(height_prob, weight_prob)`, computes the probability of sex based on the probabilities of height and weight.

    Parameters:
    - height_prob: Dictionary containing probabilities for height categories.
    - weight_prob: Dictionary containing probabilities for weight categories.

    Returns:
    - Dictionary containing probabilities for male (M) and female (F) sexes.
    '''
    # h_size = len(height_prob)
    # w_size = len(weight_prob)
    sex_dic = {'M': 0, 'F': 1}
    sex_probability = np.zeros(2)
    index = 0
    for s,p_s in sex_dic.items():
      sex_p = 0
      for h,p_h in height_prob.items():
        for w,p_w in weight_prob.items():
          sex_p += p_h * p_w * P_sex_given_hw[(s,h,w)]
      sex_probability[index] = sex_p
      index += 1
    return {'M': sex_probability[0], 'F':sex_probability[1]}


In [33]:
# Run this code to test you implementation
# Example probabilities of height
P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
# Example probabilities of weight
P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}
# Example conditional probabilities of sex given height and weight


sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)

Probability of sex: {'M': 0.6355000000000002, 'F': 0.36450000000000005}


In [34]:
# The probability of hear disease given sex, DCM and blood type, the tuple is in this order: (heart disease, sex, blood type, DCM)
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

In [35]:
def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    '''
    Function: Calculate Heart Disease Probability

    This function, `calculate_heart_disease_probability(DCM_prob, sex_prob, blood_type_prob, heart_disease_given_parents)`, computes the probability of heart disease based on the probabilities of Dilated Cardiomyopathy (DCM), sex, and blood type.

    Parameters:
    - DCM_prob: Dictionary containing probabilities for DCM categories.
    - sex_prob: Dictionary containing probabilities for sex categories.
    - blood_type_prob: Dictionary containing probabilities for blood type categories.

    Returns:
    - Dictionary containing probabilities for the presence (Y) and absence (N) of heart disease.
    '''

    heart_disease_dic = {'Y': 0, 'N': 1}

    heart_disease_probability = np.zeros(2)

    # TODO: complete the code
    index = 0
    for hd,p_hd in heart_disease_dic.items():
      hd_p = 0
      for s,p_s in sex_prob.items():
        for b,p_b in blood_type_prob.items():
          for dcm,p_dcm in DCM_prob.items():
            hd_p += p_s * p_b * p_dcm * P_heart_disease_given_parents[(hd,s,b,dcm)]
      heart_disease_probability[index] = hd_p
      index += 1

    return {'Y':heart_disease_probability[0], 'N':heart_disease_probability[1]}


In [36]:
# Run this code to test you implementation
# Example probabilities of DCM
P_DCM = {'Y': 0.2, 'N': 0.8}
# Example probabilities of blood type
P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
# Example probabilities of sex
P_sex = {'M': 0.3, 'F': 0.7}


heart_disease_probabilities = calculate_heart_disease_probability(P_DCM, P_sex, P_blood_type)
print("Probability of heart disease: ",heart_disease_probabilities)

Probability of heart disease:  {'Y': 0.41519999999999996, 'N': 0.5848}


# Loading DNA Sequences and Heart Disease Probability Calculation (20 points)

In this part, we will load the DNA sequences from the file DNA_patients.csv and calculate the probability of heart disease for each patient. Subsequently, we will classify the patients based on their heart disease probability and save the data.

In [37]:
# Read the CSV file into a DataFrame
df = pd.read_csv('DNA_patients.csv')

df.head()

,ID,DNA
0,1,CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACA...
1,2,TTGATGTAGAAGTATATTTGTTGGGTATTTGAGGTAACGTTATTAG...
2,3,ATATAATTTAAAGTCACTGGAAAAAAACAACCTAATAAAAACCACC...
3,4,TAAAGACAAAATTAAATTGAAGTAATGTTATGTTAAAATTTTGAAT...
4,5,AGCGTTTGTTCGTTAGCCGTAGGCAATGACGTGATTCAGGTCTGTG...


In [40]:
# TODO: complete the code
# Extract DNA sequences and store them in a list
DNAs = df['DNA']

states = ['H', 'L']
emissions = [{'A' : 0.2, 'C' : 0.3, 'G' : 0.3, 'T' : 0.2},
             {'A' : 0.3, 'C' : 0.2, 'G' : 0.2, 'T' : 0.3}]
start_probabilities = [0.5, 0.5]
transition_probabilities = [[0.5 , 0.5] , [0.4 , 0.6]]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

GC_content_seqs = []

for seq in DNAs:
  path, max_probability = hmm.viterbi(seq)
  GC_content_seqs.append(path)

In [44]:
# TODO: complete the code, classify the Patients as 1 if the probability of heart disease is more that 0.5 and 0 otherwise
heart_disease_classified = []
for seq in GC_content_seqs:
  height_probability, weight_probability, blood_type_probability, DCM_probability = probabilities_from_sequence(seq)
  sex_probabilities = calculate_sex_probability(height_probability, weight_probability)
  heart_disease_probabilities = calculate_heart_disease_probability(DCM_probability, sex_probabilities, blood_type_probability)
  if heart_disease_probabilities['Y'] > 0.5:
    heart_disease_classified.append(1)
  else:
    heart_disease_classified.append(0)

In [45]:
# Write the result to CSV
labels = [(i+1, heart_disease_classified[i]) for i in range(len(heart_disease_classified))]
csv_filename = "heart_disease_result.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label'])
    for id, label in labels:
        writer.writerow([id, label])

**Note: You should upload heart_disease_result.csv along side your notebook.**